In [27]:
import pandas as pd
import numpy as np

## State CDC Dataframe creation

In [28]:
df_states = pd.read_csv("data/State_Custom_Data.csv")

In [29]:
#CDC will add 'Insufficient Data' as a value in columns
df_states = df_states[df_states['TOTAL DEATHS'] != 'Insufficient Data']

In [30]:
#CDC data has commas in numbers
df_states = df_states.replace(to_replace =',', value = '', regex = True) 


In [31]:
#rename columns to remove spaces
column_list = df_states.columns
new_column_names = []
for column in column_list:
    new_column_name = column.capitalize().replace(' ','_').replace('&','_')
    new_column_names.append(new_column_name)

df_states.columns  = new_column_names

In [32]:
#convert appropriate columns to ints and floats
df_states[['Num_influenza_deaths','Num_pneumonia_deaths','Total_deaths']] = \
df_states[['Num_influenza_deaths','Num_pneumonia_deaths','Total_deaths']].astype(int)

df_states['Percent_p_i'] = df_states['Percent_p_i']/100
# df_states['Threshold'] = df_states['Threshold']/100
# df_states['Baseline'] = df_states['Baseline']/100

df_states['Percent_complete'] = df_states['Percent_complete'].str.replace('> 100%', '100%', regex=False)
df_states['Percent_complete'] = df_states['Percent_complete'].str.rstrip('%').astype('float') / 100.0

In [33]:
#add Calendar_year column
conditions = [
    df_states['Week']<40,
    df_states['Week']>=40    
]

df_states['First_year'] = df_states['Season'].apply(lambda x: int(x[0:4])) 
df_states['Second_year'] = df_states['Season'].apply(lambda x: int(x[0:4])+1) 
choices = [df_states['Second_year'],df_states['First_year']]
df_states['Calendar_year'] = np.select(conditions,choices)
df_states['Calendar_year'] = df_states['Calendar_year'].astype(int) 

In [34]:
#drop unused columns
df_states = df_states.drop('First_year', axis = 1)
df_states = df_states.drop('Second_year', axis = 1)
# df_states = df_states.drop('Sub_area', axis = 1)
df_states = df_states.drop('Age_group', axis = 1)


                 

In [35]:
#sort values and reset index
df_states['State'] = df_states['Sub_area']
df_states = df_states.sort_values(['State','Calendar_year','Week']).reset_index()
df_states = df_states.drop('index', axis = 1)


In [36]:

df_states = df_states.drop('Sub_area', axis = 1)
df_states = df_states.drop('Area', axis = 1)

In [37]:
df_states

,Season,Week,Percent_p_i,Num_influenza_deaths,Num_pneumonia_deaths,Total_deaths,Percent_complete,Calendar_year,State
0,2012-13,1,0.088,4,73,874,0.900,2013,Alabama
1,2012-13,2,0.093,12,100,1200,1.000,2013,Alabama
2,2012-13,3,0.100,8,102,1105,1.000,2013,Alabama
3,2012-13,4,0.098,4,100,1060,1.000,2013,Alabama
4,2012-13,5,0.083,2,83,1026,1.000,2013,Alabama
...,...,...,...,...,...,...,...,...,...
19582,2019-20,8,0.058,0,6,104,1.000,2020,Wyoming
19583,2019-20,9,0.037,0,4,108,1.000,2020,Wyoming
19584,2019-20,10,0.125,1,9,80,0.891,2020,Wyoming
19585,2019-20,11,0.049,0,4,82,0.914,2020,Wyoming


In [38]:
#bring in Population Dataframe
df_population = pd.read_csv("data/population_df.csv", index_col=0)

In [39]:
#Merge National and Population DataFrame to get Population
df_states_official = pd.merge(df_states, df_population,  how='left', 
                                left_on=['State','Calendar_year'], right_on = ['Name','Calendar_year'])



In [40]:
# Drop useless col and add Deaths per hund thou metric
df_states_official = df_states_official.drop('Name', axis = 1)
df_states_official['Deaths_per_hund_thou'] = df_states_official['Total_deaths'] / (df_states_official['Population']/100000)

In [42]:
#save dataframe as csv file to be used in other files
df_states_official.to_csv('data/states_cdc_pandas_df.csv')